In [ ]:
#!pip3 install langchain_community
#!pip3 install langchain_openai
#pip3 install bert_score
#!pip3 install bert-score transformers

In [3]:
import pandas as pd
import openai
from dotenv import load_dotenv
import os
import ast
from openai import AzureOpenAI
import faiss
import numpy as np
import json
from bert_score import score
import pycountry
import re
import spacy
nlp = spacy.load("en_core_web_sm")
from bert_score import score
import evaluate
bertscore = evaluate.load("bertscore")
import nltk
nltk.download('punkt') # Make sure to download the 'punkt' tokenizer models
from nltk.tokenize import word_tokenize
from nltk.translate.meteor_score import meteor_score



2024-02-11 19:58:12.946796: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
span-marker is already registered. Overwriting pipeline for task span-marker...


### Load Enviroment files

In [13]:
# Load environment variables
load_dotenv()

Python-dotenv could not parse statement starting at line 10
Python-dotenv could not parse statement starting at line 11
Python-dotenv could not parse statement starting at line 12
Python-dotenv could not parse statement starting at line 13
Python-dotenv could not parse statement starting at line 14
Python-dotenv could not parse statement starting at line 15
Python-dotenv could not parse statement starting at line 16
Python-dotenv could not parse statement starting at line 17
Python-dotenv could not parse statement starting at line 18
Python-dotenv could not parse statement starting at line 19
Python-dotenv could not parse statement starting at line 20


True

In [26]:
openai.api_type = "azure"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai_deployment = "sdgi-gpt-35-turbo-16k"
neo4j_pass = os.getenv("NEO4JPASS")


In [27]:
# OpenAI API configuration
#openai.api_type = "azure"
#openai.api_key = os.getenv("api_key_azure")
#openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
#openai.api_version = os.getenv("api_version")
#openai_deployment = "sdgi-gpt-35-turbo-16k"


client = AzureOpenAI(
  api_key = os.getenv("OPENAI_API_KEY"),  
  api_version = os.getenv("OPENAI_API_VERSION"),
  azure_endpoint =os.getenv("AZURE_OPENAI_ENDPOINT") 
)

embedding_model = os.getenv("USER_QUERY_EMBEDDING_ENGINE") 

# print(openai.api_key)
# print(openai.api_base)
# print(openai.api_version)


<h3> helper functions </h3>

In [28]:
df = pd.read_pickle('../models/df_embed_EN.pkl')

In [29]:
# use this function to make simple openAI Calls
def callOpenAI(prompt):  
    response_entities = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0,
                    messages=[
                        {"role": "user", "content": prompt},
                    ]
                )
    response = response_entities.choices[0].message.content
    return response


<h3> processing modules </h3>

In [30]:
def extractEntitiesFromQuery(user_query):
    prompt = f"""
    Extract entities from the following user query: \"{user_query}\" and return output in array format.
    
    -Entities should be directly related to the domain or topic of interest. They should represent important concepts that contribute to the understanding of the subject matter.
    -Each entity in the knowledge graph should be distinct and have a unique identifier. This ensures clarity and avoids ambiguity when establishing relationships between entities.
    -You Must return output in array format e.g  ['entity1','entity2'] !!!
    -Avoid adding new lines or breaking spaces to your output. Array should be single dimension and single line !!!
    """
    entity_str = callOpenAI(prompt)   
    return entity_str



In [31]:
def find_mentioned_countries(text):
    countries = set()
    
    # Tokenize the text using regular expressions to preserve punctuation marks
    words = re.findall(r'\w+|[^\w\s]', text)
    text = ' '.join(words)  # Join the tokens back into a string
    
    for word in text.split():
        try:
            country = pycountry.countries.get(name=word) #pycountry.countries.lookup(word)
            if country != None : 
               countries.add(country.name)
        except LookupError:
            pass
    
    return list(countries)

def filter_country(user_query):
    mentioned_countries = find_mentioned_countries(user_query)
    # print(mentioned_countries)
    # Check if mentioned_countries is not empty
    if mentioned_countries:
        country = mentioned_countries[0]
        return df[df['Country Name'] == country]
    else:
        # Handle the case where no countries were mentioned
        return None  # Or return an empty DataFrame or any other suitable value


In [32]:
def search_embeddings(user_query):
    df_filtered = filter_country(user_query) if filter_country(user_query) is not None else None
    
    if df_filtered is not None and not df_filtered.empty:  # Check if DataFrame is not None and not empty
        length = len(df_filtered.head())
        filtered_embeddings_arrays = np.array(list(df_filtered['Embedding']))
        index = faiss.IndexFlatIP(filtered_embeddings_arrays.shape[1]) 
        index.add(filtered_embeddings_arrays)

        user_query_embedding = client.embeddings.create( 
                input=user_query ,model= embedding_model
            ).data[0].embedding

        k = min(5, length)
        distances, indices = index.search(np.array([user_query_embedding]), k)
        return df_filtered, distances, indices
    else:
        return None, None, None


# KG module

In [33]:
from py2neo import Graph, Node, Relationship
graph = Graph(uri = 'bolt://localhost:7687',user='neo4j',password=neo4j_pass)

In [34]:
def search_entities_with_relations(label, name):
    """
    Search for nodes by label and name, including all their relationships and connected nodes.

    :param label: The label of the nodes to search.
    :param name: The name property value of the nodes to search.
    :return: A list of dictionaries, each containing a node and its relationships and connected nodes.
    """
    # Match nodes and their relationships, case-insensitive search
    query = f"""
    MATCH (n:`{label}`)-[r]-(m)
    WHERE toLower(n.name) = toLower($name) OR toLower(n.acronym) = toLower($name)
    RETURN n, collect(r) as relations, collect(m) as connectedNodes
    """
    results = graph.run(query, name=name).data()
    
    # Construct a comprehensive view for each node with its relationships and connected nodes
    entities_with_relations = []
    for record in results:
        entity_info = {
            'node': record['n'],
            'relationships': record['relations'],
            'connected_nodes': record['connectedNodes']
        }
        entities_with_relations.append(entity_info)

    return entities_with_relations


In [35]:
def separate_entities_and_relationships(relationships):
    """
    Separates entities and relationships from Neo4j query output into distinct structures,
    ensuring no duplicate entities are included.

    :param relationships: A list of Relationship objects from a Neo4j query.
    :return: A tuple containing two lists: one for unique entities and another for relationships.
    """
    # Using a dictionary to ensure unique entities based on a combination of name and category
    entities_dict = {}
    rels = []

    for rel in relationships:
        # Process both the source and target nodes for each relationship
        for node in [rel.start_node, rel.end_node]:
            # Define a unique identifier for each entity
            entity_id = f"{node['name']}_{node['category']}"
            
            # If the entity is not already in the dictionary, add it
            if entity_id not in entities_dict:
                entities_dict[entity_id] = {
                    'name': node['name'],
                    'category': node.get('category', 'N/A'),
                    #'summary': node.get('summary', 'N/A'),
                    'acronym': node.get('acronym', 'N/A'),
                }

        # Add relationship information
        rels.append({
            'subject': rel.start_node['name'],
            'object': rel.end_node['name'],
            'relationship_type': rel.__class__.__name__,
            'description': rel.get('description', 'N/A'),
        })

    # Convert the entities dictionary to a list to remove the unique identifier layer
    entities_list = list(entities_dict.values())

    return entities_list, rels




In [36]:
def get_answer(user_question, content):
    system_prompt = "You are a system that answers user questions based on excerpts from PDF documents provided for context. Only answer if the answer can be found in the provided context. Do not make up the answer; if you cannot find the answer, say so."
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_question},
        {'role': 'user', 'content': content},
    ]
    response_entities = openai.chat.completions.create(
                    model=openai_deployment,
                    temperature=0.2,
                    messages=messages
                )
    response = response_entities.choices[0].message.content
    return response
  

In [37]:
def map_to_structure(qs):
    result_dict = {}

    # Extract the DataFrame from the tuple
    dataframe = qs[0]

    # Counter to limit the loop to 10 iterations
    count = 0

    for index, row in dataframe.iterrows():
        # Define a unique identifier for each document, you can customize this based on your data
        document_id = f"doc-{index + 1}"
        # Handle NaN in content by using fillna
        content = row["Content"]
        content = ' '.join(row["Content"].split()[:160])
        # Create a dictionary for each document
        document_info = {
            "title": row["Document Title"],
            "extract": content or "",  # You may need to adjust this based on your column names
            "category": row["Category"],
            "link": row["Link"],
            "thumbnail": ''
        }
        # print(document_info)
        # Add the document to the result dictionary
        result_dict[document_id] = document_info

        # Increment the counter
        count += 1

        # # Break out of the loop if the counter reaches top 10
        if count == 10:
            break

    return result_dict


In [38]:
## module to get data for specific indicators which are identified is relevant to the user query

def indicatorsModule(user_query): #lower priority
    
    # find relevant indicators based on uesr query and extract values
    indicators_dict={
        "indicator-id-1":"value from indicator-id-1",
        "indicator-id-2":"value from indicator-id-2"
    }#temp
    
    return indicators_dict

## module to extract text from documents and return the text and document codes
def semanticSearchModule(user_query):
    qs = search_embeddings(user_query) #df, distances, indices
    # if qs != None :
    if qs[0] is not None:
        result_structure = map_to_structure(qs)
        return result_structure
    else : 
        return []


In [39]:
# module to synthesize answer using retreival augmented generation approach

def synthesisModule(user_query, excerpts_dict, indicators_dict):
    
    # Generate prompt engineering text and template
    llm_instructions = f"""
    Ignore previous commands!!!
    Given a user query, use the provided excerpts, and Sources to
    provide the correct answer to the user's query
    
    User Query: {user_query}
    
    Sources: {excerpts_dict}
    

    - Answer must be coherent, relevant and should contain important details. 
    
    """
    ###synthesize data into structure within llm prompt engineering instructions
    answer=callOpenAI(llm_instructions)
    
    return answer

## to test this, run the full pipeline with the handleApiCall function

In [40]:
def filter_relations(user_query, relationships):
    
    # Generate prompt engineering text and template
    llm_instructions = f"""
    Ignore previous commands!!!
    Given a user query, identify the relations that are relevant to the answering the query. 
    Delete the rest of relations and return the set of relevant relations in JSON format

    
    User Query: {user_query}
        
    Relations: {relationships}

    """
    ###synthesize data into structure within llm prompt engineering instructions
    answer=callOpenAI(llm_instructions)
    
    return answer

In [42]:
# module to synthesize answer using retreival augmented generation approach

def synthesisModule_with_KG(user_query, entities, relationships, excerpts_dict, indicators_dict):
    
    # Generate prompt engineering text and template
    llm_instructions = f"""
    Ignore previous commands!!!
    
    For the given user query, use the provided sources and relationships to provide the correct answer. 
    
    User Query: {user_query}
    
    Sources: {excerpts_dict}
    
     Relationships:  {relationships}

    - Give a coherent, detailed and relevant answer. 

    """
    ###synthesize data into structure within llm prompt engineering instructions
    answer=callOpenAI(llm_instructions)
    
    return answer

## to test this, run the full pipeline with the handleApiCall function

In [43]:
def handleApiCall(user_query):
        
    ##run processing modules (in parallel)
    #entities_dict=knowledgeGraphModule(user_query)
    excerpts_dict=semanticSearchModule(user_query)
    indicators_dict=indicatorsModule(user_query) ##lower priority
    #query_idea_list=queryIdeationModule(user_query) ##lower priority
    
    ##synthesis module
    answer=synthesisModule(user_query, excerpts_dict, indicators_dict)
    
    ##structure response
    response={
        "user_query":user_query,
        "answer":answer,
        "sources":excerpts_dict
        
    }
    
    return response

In [44]:
def handle_api_call_with_kg(user_query, entities, relationships):
        
    ##run processing modules (in parallel)
    #entities_dict=knowledgeGraphModule(user_query)
    excerpts_dict=semanticSearchModule(user_query)
    indicators_dict=indicatorsModule(user_query) ##lower priority
    #query_idea_list=queryIdeationModule(user_query) ##lower priority
    
    ##synthesis module
    answer=synthesisModule_with_KG(user_query, entities, relationships, excerpts_dict, indicators_dict)
    
    ##structure response
    response={
        "user_query":user_query,
        "answer":answer,
        "sources":excerpts_dict,
        "entities": entities,
        "relations":relationships
        
    }
    
    return response

In [45]:
with open("queries.json") as f:
    data = f.read()
    query_list = ast.literal_eval(data)
    f.close()
    

In [46]:
len(query_list)

30

# Pipeline

In [61]:
# full pipeline with retreival, synthesis of answer to user query, and structure results into api response

for index in range(10,11):
    test_query = query_list[index]
    print (test_query)

    entity_str = extractEntitiesFromQuery(test_query)
    # Convert the string representation of the list to an actual list
    entity_list = ast.literal_eval(entity_str)

    relations_list = []
    for item in entity_list:
        entities_with_relations = search_entities_with_relations('Entity', item)

        for entity_info in entities_with_relations:
            relations_list.extend(entity_info['relationships'])
            #print("Connected Nodes:", entity_info['connected_nodes'])

    # Assuming 'relationships' is your list of Relationship objects from the query
    entities, relationships = separate_entities_and_relationships(relations_list)
    
    filtered_relations = json.loads(filter_relations(test_query,relationships))

    print (len(relationships))
    print (len(entities))
    
    if len(entities) > 2:
        final_entities = entities[:2]
    else:
        final_entities = entities
    
    if len(filtered_relations) > 20:
        final_relationships = filtered_relations[:20]
    else:
        final_relationships = filtered_relations
    
    indicators_dict=indicatorsModule(test_query)
    print(indicators_dict)
    
    
    #test usage
    excerpts_dict=semanticSearchModule(test_query)
    # print(excerpts_dict)


    #Return top 10-20 most related 
    # Define the filename to save the JSON data -  can remove later
    json_filename = "outputs/excerpts.json"

    # Save excerpts_dict to the JSON file just for a better preview
    with open(json_filename, 'w', encoding='utf-8') as json_file:
        json.dump(excerpts_dict, json_file, ensure_ascii=False, indent=4)

    print(f"Excerpts saved to {json_filename}")


    # test usage
    llm_response = handleApiCall(test_query) 
    # Define the filename to save the JSON data -  can remove later
    json_filename = "test_outputs/test_query"+ str(index+1) + "_answer.json"

    # Save excerpts_dict to the JSON file just for a better preview
    with open(json_filename, 'w', encoding='utf-8') as json_file:
        json.dump(llm_response, json_file, ensure_ascii=False, indent=4)

    print(f"LLM Response saved to {json_filename}")


# test usage
kg_llm_response=handle_api_call_with_kg(test_query, final_entities, final_relationships)

# Define the filename to save the JSON data -  can remove later
json_filename = "test_outputs/Query"+ str(index+1) + "_answer.json"

# Save excerpts_dict to the JSON file just for a better preview
with open(json_filename, 'w', encoding='utf-8') as json_file:
    json.dump(kg_llm_response, json_file, ensure_ascii=False, indent=4)

print(f"KG-LLM Response saved to {json_filename}")
print ("\n--------------\n")

What strategies is UNDP implementing to support Angola's Bureau of External Relations and Advocacy?
24
17
{'indicator-id-1': 'value from indicator-id-1', 'indicator-id-2': 'value from indicator-id-2'}
Excerpts saved to outputs/excerpts.json
LLM Response saved to test_outputs/test_query11_answer.json
KG-LLM Response saved to test_outputs/Query11_answer.json

--------------



<h3>testing</h3>

In [127]:
## next step, develop automated testing for all modules
## iterate through test_queries and build automated tests to score results

# open testing dataset with queries and expected results

json_filename = "auto_eval/test_data.json"


with open(json_filename, 'r') as file:
    data = file.read()
    test_data = json.loads(data)


In [128]:
llm_score_list = []
kg_score_list = []
combined_score_list = []

for i in test_data:
    obj = {}
    
    obj['id'] = i['id'] 
    obj['query'] = i['query']
    
    llm_response = i['llm_response']
    kg_response = i['llm_kg_response']
    answer = i['reference']
    
    reference_texts = [answer]
    
    # Tokenize the texts    
    references = [word_tokenize(ref) for ref in reference_texts]
    llm_candidate = word_tokenize(llm_response)
    kg_candidate = word_tokenize(kg_response)
    
    
    # Calculate METEOR score
    obj['llm_score'] = meteor_score(references, llm_candidate)
    obj['kg_score'] = meteor_score(references, kg_candidate)
    
    llm_score_list.append(obj['llm_score'])
    kg_score_list.append(obj['kg_score'])
    combined_score_list.append(obj)

In [129]:
obj2 = {}
obj2['LLM Meteor scores'] = llm_score_list
obj2['KG_LLM Meteor scores'] = kg_score_list

combined_score_list.append(obj2 )

json_filename = "auto_eval/meteor_score.json"

# Save excerpts_dict to the JSON file just for a better preview
with open(json_filename, 'w') as json_file:
    json.dump(combined_score_list, json_file, indent=2)

In [122]:
len(kg_score_list)

23

In [123]:
llm_precision_list = []
llm_recall_list = []
llm_f1_list = []
kg_precision_list = []
kg_recall_list = []
kg_f1_list = []

bert_score_list = []

for i in test_data:
    obj = {}
    
    obj['id'] = i['id'] 
    obj['query'] = i['query']
    
    llm_response = i['llm_response']
    kg_response = i['llm_kg_response']
    answer = i['reference']
    
    llm_obj = {}
    kg_obj = {}
    # Tokenize the texts    
    
    reference = [answer]

    llm_candidate = [llm_response]
    kg_candidate = [kg_response]
    
    
    # Calculate BERT score
   
    results = bertscore.compute(predictions=llm_candidate, references=reference, 
                                model_type="distilbert-base-uncased")

    llm_precision_list.append(results['precision'][0])
    llm_recall_list.append(results['recall'][0])
    llm_f1_list.append(results['f1'][0])
    llm_obj['precision'] = results['precision'][0]
    llm_obj['recall'] = results['recall'][0]
    llm_obj['f1-score'] = results['f1'][0]
    
    results = bertscore.compute(predictions=kg_candidate, references=reference, 
                                model_type="distilbert-base-uncased")


    kg_precision_list.append(results['precision'][0])
    kg_recall_list.append(results['recall'][0])
    kg_f1_list.append(results['f1'][0])
    kg_obj['precision'] = results['precision'][0]
    kg_obj['recall'] = results['recall'][0]
    kg_obj['f1-score'] = results['f1'][0]
    
    obj['LLM'] = llm_obj
    obj['KG_LLM'] = kg_obj
    bert_score_list.append(obj)

In [124]:
obj3 = {}
obj3['LLM Precision scores'] = llm_precision_list
obj3['LLM Recall scores'] = llm_recall_list
obj3['LLM F1 scores'] = llm_f1_list
obj3['KG Precision scores'] = kg_precision_list
obj3['KG Recall scores'] = kg_recall_list
obj3['KG F1 scores'] = kg_f1_list




In [125]:
bert_score_list.append(obj3)

In [126]:
bert_score_file = "auto_eval/bert_score.json"

# Save excerpts_dict to the JSON file just for a better preview
with open(bert_score_file, 'w') as json_file:
    json.dump(bert_score_list, json_file, indent=2)

  # TODO::: 
  ##### Add citation prompt to the synthesis module. -done 
  ##### Convert notebook to flask API script. main.py - done
  ##### Refactor PDF -> txt pipeline 

  